In [118]:
import pandas as pd
import os
import re

In [135]:
# Definimos los paths a las carpetas qeu contienen los datos de las boyas
paths = ["boyasRevisadas/BoyasUPCT/", "boyasRevisadas/Boyasprof/"]

# Definimos un diccionario en el que ir guardando en dataframes de cada fichero
dataframes_boyas = {}

# Diccionario de traducciones
nombres_es = {
    "Chlorophyll": "Clorofila",
    "Temperature": "Temperatura",
    "Dissolvedoxygen": "OxigenoDisuelto",
    "Salinity": "Salinidad",
    "Turbidity": "Turbidez"
}

# Para activar/desactivar nombres en español
usar_es = True  # cambia a False para mantener los nombres en inglés

# Iteramos para los paths
for path in paths: 
    # Cogemos el identificador de las boyas
    buoy_ids = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    # Iteramos sobre esos identificadores
    for buoy_id in buoy_ids:
        # Listamos los csv
        archivos_csv = [f for f in os.listdir(path + buoy_id) if f.endswith('.csv')]
        # Iteramos sobre los csv
        for archivo in archivos_csv:
            # Cogemos el nombre de la variable
            nombre_variable = os.path.splitext(archivo)[0]
            # Y definimos su ruta completa
            ruta_completa = os.path.join(path, buoy_id, archivo)
            # Condición para que si alguna de las propiedades de interés está en el nombre de la variable 
            if any(substr in nombre_variable for substr in ["Chlorophyll", "Temperature", "Dissolved oxygen", "Salinity", "Turbidity"]):
                nombre_variable = nombre_variable.replace(" ", "")

                # Traducción
                if usar_es:
                    for en, es in nombres_es.items():
                        if en.replace(" ", "") in nombre_variable:
                            nombre_variable = nombre_variable.replace(en.replace(" ", ""), es)
                            break

                # Cargamos csv (con separador por ; e ignorando las primeras filas, que contienen información adicional) y lo guardamos en el diccionario de dataframes
                dataframes_boyas[f"{buoy_id}_{nombre_variable}"] = pd.read_csv(ruta_completa, skiprows=5, sep=';') 

# Para cada uno de los dataframes
for key, df in dataframes_boyas.items():
    try:
        # Cogemos la fecha y la ponemos en formato datetime
        df['Date'] = pd.to_datetime(df['Date'])
    except:
        print(key)

In [142]:
# dataframes_boyas: dict con claves tipo 'E3_Temperature_E3_UPCT' y valores DataFrames
# Cada DF tiene columnas: 'Date', 'Depth_-0.5m', 'Depth_-1m', ... (las que existan)

# Expresión regular para la columna de profundidad
reg_depth = re.compile(r'^Depth_(-?\d+(?:\.\d+)?)m$')

# Lista vacía para ir guardando filas
registros = []

for key, df in dataframes_boyas.items():
    print(key)
    # 1) Extrae la variable del nombre de la clave
    #    Asumiendo estructura 'BOYA_Variable_...'
    partes = key.split('_')
    if len(partes) < 2 or 'Date' not in df.columns:
        continue
    # Extraemos el nombre de la variable (Chl, Temp, OD, etc)
    variable = partes[1]

    # 2) Aseguramos fechas como datetime
    dfi = df.copy()
    dfi['Date'] = pd.to_datetime(dfi['Date'], errors='coerce')
    dfi = dfi.dropna(subset=['Date'])

    # 3) Selecciona columnas de profundidad tipo 'Depth_-0.5m'
    depth_cols = [c for c in dfi.columns if reg_depth.match(c)]

    if not depth_cols:
        continue

    # 4) Pasa a formato largo - para poder hacer operaciones por variable y por profundidad
    # Así tendremos solamente estas cuatro columnas
    long = dfi.melt(
        id_vars='Date',
        value_vars=depth_cols,
        var_name='depth_col',
        value_name='value'
    )

    # 5) Extrae la profundidad numérica
    long['depth'] = long['depth_col'].str.extract(r'Depth_(-?\d+(?:\.\d+)?)m').astype(float)

    # Asignar variable
    long['variable'] = variable

    # 6) Guardamos filas válidas
    registros.append(long[['Date', 'variable', 'depth', 'value']])

# 7) Concatena todo
if registros:
    datos = pd.concat(registros, ignore_index=True)

    # 8) Agrega por (variable, profundidad, fecha):
    #    si hay múltiples muestras ese día (por la misma o distintas boyas), saca la media.
    try:
        agregada = (
            datos
            .groupby(['variable', 'depth', 'Date'], as_index=False, observed=True)['value']
            .mean().round(2)
        )
    except Exception as e:
        print(f"Error al agrupar en {key}: {type(e).__name__} → {e}")

    # Profundidades que usamos como columnas
    depths_obj = [-0.5, -1.0, -1.5, -2.0, -2.5, -3.0, -3.5, -4.0, -4.5]

    # Crear un DataFrame por variable
    dfs_por_variable = {}
    for var, df_var in agregada.groupby('variable'):
        # Para volver al formato ancho pero habiendo hecho la agregación, y las fechas pasan a ser el índice, y cada profundidad será una colmna
        piv = df_var.pivot(index='Date', columns='depth', values='value')

        # 1) Unión de fechas disponibles para esa variable (de la tabla ya agregada)
        fechas_union = pd.Index(sorted(df_var['Date'].unique()))
        #Reindex para garantizar que aparezcan todas las fechas
        piv = piv.reindex(index=fechas_union)

        # 2) Reindexa columnas a las profundidades objetivo: las que falten quedarán como NaN
        piv = piv.reindex(columns=depths_obj)

        piv.index.name = 'Date'  
        df_final = piv.reset_index()

        # 3) Ordenamos las columnas como: -0.5, -1.0, ...
        # (ya están en ese orden; si no, reordena)
        piv = piv[depths_obj]

        # 4) Quitamos nombres de ejes y convierte Date a columna
        piv.columns.name = None
        df_final = piv.reset_index()  # primera col: Date

        dfs_por_variable[var] = df_final

        # Guardamos un csv por variable:
        df_final.to_csv(f"boyasRevisadas/{var}_series.csv", index=False)

else:
    dfs_por_variable = {}


E3_Turbidez_E3_UPCT
E3_Temperatura_E3_UPCT
E3_Clorofila_E3_UPCT
E3_OxigenoDisuelto_E3_UPCT
E3_Salinidad_E3_UPCT
E12_Temperatura_E12_UPCT
E12_Salinidad_E12_UPCT
E12_Clorofila_E12_UPCT
E12_Turbidez_E12_UPCT
E12_OxigenoDisuelto_E12_UPCT
E9_Clorofila_E9_UPCT
E9_OxigenoDisuelto_E9_UPCT
E9_Salinidad_E9_UPCT
E9_Turbidez_E9_UPCT
E9_Temperatura_E9_UPCT
E8_Clorofila_E8_UPCT
E8_Turbidez_E8_UPCT
E8_Temperatura_E8_UPCT
E8_OxigenoDisuelto_E8_UPCT
E8_Salinidad_E8_UPCT
E7_Turbidez_E7_UPCT
E7_Temperatura_E7_UPCT
E7_Clorofila_E7_UPCT
E7_Salinidad_E7_UPCT
E7_OxigenoDisuelto_E7_UPCT
E4_Temperatura_E4_UPCT
E4_Turbidez_E4_UPCT
E4_OxigenoDisuelto_E4_UPCT
E4_Clorofila_E4_UPCT
E4_Salinidad_E4_UPCT
E2_Turbidez_E2_UPCT
E2_Temperatura_E2_UPCT
E2_Salinidad_E2_UPCT
E2_OxigenoDisuelto_E2_UPCT
E2_Clorofila_E2_UPCT
E10_Turbidez_E10_UPCT
E10_Temperatura_E10_UPCT
E10_Salinidad_E10_UPCT
E10_OxigenoDisuelto_E10_UPCT
E10_Clorofila_E10_UPCT
E6_OxigenoDisuelto_E6_UPCT
E6_Clorofila_E6_UPCT
E6_Turbidez_E6_UPCT
E6_Salinidad_E6_

In [143]:
dfs_por_variable["Salinidad"]

,Date,-0.5,-1.0,-1.5,-2.0,-2.5,-3.0,-3.5,-4.0,-4.5
0,2016-08-02,45.93,45.57,NaN,45.57,NaN,NaN,NaN,NaN,NaN
1,2016-08-04,46.38,46.00,NaN,45.92,NaN,NaN,NaN,NaN,NaN
2,2016-08-09,46.34,45.79,NaN,45.72,NaN,NaN,NaN,NaN,NaN
3,2016-08-25,46.50,46.42,NaN,46.40,NaN,NaN,NaN,NaN,NaN
4,2016-08-30,46.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
420,2024-06-18,43.34,43.28,43.53,43.80,43.84,43.91,43.93,43.93,43.64
421,2024-06-25,42.78,44.03,43.91,44.08,44.11,44.52,44.74,44.17,43.97
422,2024-07-03,43.39,43.49,43.59,43.58,43.93,44.27,44.32,44.31,44.30
423,2024-07-09,43.99,43.81,44.16,43.34,43.64,43.82,43.93,44.02,44.02
